In [83]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [84]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\88019\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [85]:
print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

In [86]:
df_Fake = pd.read_csv("Fake.csv")
df_Fake.head()


,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [87]:
df_True = pd.read_csv("True.csv")
df_True.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [88]:
df_Fake["label"] = 0
df_True["label"] = 1

In [89]:
news = pd.concat([df_Fake, df_True], axis = 0)

In [90]:
news.head()


,title,text,subject,date,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0


In [91]:
news.tail()

,title,text,subject,date,label
21412,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,"August 22, 2017",1
21413,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017",1
21414,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017",1
21415,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017",1
21416,Indonesia to buy $1.14 billion worth of Russia...,JAKARTA (Reuters) - Indonesia will buy 11 Sukh...,worldnews,"August 22, 2017",1


In [92]:
news = news.sample(frac=1)

In [93]:
news.head()

,title,text,subject,date,label
224,Trump Administration Tries To Fix His Dumbest...,Donald Trump s administration is probably the ...,News,"October 13, 2017",0
13418,SHOCKING SUMMARY Of The DNC Convention So Far…...,What a crazy group of professional agitators a...,politics,"Jul 26, 2016",0
14807,Alaska Airlines discontinues Los Angeles-Havan...,(Reuters) - U.S. airline Alaska Airlines on Tu...,worldnews,"November 15, 2017",1
7202,Minnesota Republican: Gay People Are Mentally...,He probably believes pray the gay away is th...,News,"March 30, 2016",0
14875,WATCH VIDEO: Russia Details Turkey’s Support O...,This report explains so much could this explai...,politics,"Nov 28, 2015",0


In [94]:
news.reset_index(inplace=True)
news.drop(["index"], axis = 1, inplace = True)

In [95]:
news.head()

,title,text,subject,date,label
0,Trump Administration Tries To Fix His Dumbest...,Donald Trump s administration is probably the ...,News,"October 13, 2017",0
1,SHOCKING SUMMARY Of The DNC Convention So Far…...,What a crazy group of professional agitators a...,politics,"Jul 26, 2016",0
2,Alaska Airlines discontinues Los Angeles-Havan...,(Reuters) - U.S. airline Alaska Airlines on Tu...,worldnews,"November 15, 2017",1
3,Minnesota Republican: Gay People Are Mentally...,He probably believes pray the gay away is th...,News,"March 30, 2016",0
4,WATCH VIDEO: Russia Details Turkey’s Support O...,This report explains so much could this explai...,politics,"Nov 28, 2015",0


In [96]:
news = news.drop(["title", "subject", "date"], axis = 1)

In [97]:
news.head()

,text,label
0,Donald Trump s administration is probably the ...,0
1,What a crazy group of professional agitators a...,0
2,(Reuters) - U.S. airline Alaska Airlines on Tu...,1
3,He probably believes pray the gay away is th...,0
4,This report explains so much could this explai...,0


In [98]:
news.tail()

,text,label
44893,The president prides himself on the fact that...,0
44894,21st Century Wire says Rich men love to gamble...,0
44895,"By nearly all accounts, Donald Trump s respons...",0
44896,To say that Donald Trump is a divisive figure ...,0
44897,WASHINGTON (Reuters) - A U.S. State Department...,1


In [99]:
def wordopt(text):
    text = text.lower()
    text = re.sub(r'https?://\S+|www\.\S+',"",text)

    text = re.sub(r'<.*?>','',text)

    text = re.sub(r'[^\w\s]','',text)
    text = re.sub(r'\d', '', text)
    text = re.sub(r'\n','',text)
    return text

In [100]:
news['text'] = news['text'].apply(wordopt)

In [102]:
news['text']

0        donald trump s administration is probably the ...
1        what a crazy group of professional agitators a...
2        reuters  us airline alaska airlines on tuesday...
3        he probably believes  pray the gay away  is th...
4        this report explains so much could this explai...
                               ...                        
44893     the president prides himself on the fact that...
44894    st century wire says rich men love to gamble o...
44895    by nearly all accounts donald trump s response...
44896    to say that donald trump is a divisive figure ...
44897    washington reuters  a us state department vide...
Name: text, Length: 44898, dtype: object

In [103]:
news.head()

,text,label
0,donald trump s administration is probably the ...,0
1,what a crazy group of professional agitators a...,0
2,reuters us airline alaska airlines on tuesday...,1
3,he probably believes pray the gay away is th...,0
4,this report explains so much could this explai...,0


In [104]:
#separating the data and label
X = news['text'].values
Y = news['label'].values

In [105]:
# converting the textual data to numerical data
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)

In [106]:
print(X)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 9219155 stored elements and shape (44898, 208652)>
  Coords	Values
  (0, 2414)	0.16646588140807236
  (0, 7223)	0.03128465080170592
  (0, 7288)	0.036338357335751074
  (0, 7751)	0.01700339332399992
  (0, 7914)	0.11397854718694216
  (0, 9940)	0.04867179361188072
  (0, 10633)	0.07161527316019697
  (0, 11553)	0.07924421355216306
  (0, 12405)	0.016755581850963052
  (0, 12815)	0.09508428467174318
  (0, 12845)	0.07375093777849227
  (0, 16477)	0.06609539190346408
  (0, 16690)	0.02599181621446173
  (0, 20286)	0.07269537224653741
  (0, 20805)	0.0988439731930481
  (0, 23001)	0.10883808105511293
  (0, 25091)	0.01810213754157288
  (0, 25256)	0.015782719128453575
  (0, 31146)	0.050884760177024986
  (0, 32684)	0.08371163125283511
  (0, 33647)	0.04576749878357469
  (0, 34511)	0.07010931911201446
  (0, 37131)	0.04028527286042202
  (0, 37334)	0.13666654288873656
  (0, 37937)	0.04012463155024924
  :	:
  (44897, 172845)	0.06304837211329463
  (44

In [107]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify=Y, random_state=2)

In [108]:
model = LogisticRegression()

In [109]:
model.fit(X_train, Y_train)

LogisticRegression()

In [110]:
# accuracy score on the training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [111]:
print('Accuracy score of the training data : ', training_data_accuracy)

Accuracy score of the training data :  0.9931510663177237


In [112]:
# accuracy score on the test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [113]:
print('Accuracy score of the test data : ', test_data_accuracy)

Accuracy score of the test data :  0.989532293986637
